In [ ]:
from bs4 import BeautifulSoup 
import requests

In [ ]:
# Assign URL 
url = "https://asn.flightsafety.org/database/engines/"

# Make a GET request to fetch the raw HTML content 
html_content = requests.get(url).text 


In [ ]:
# Parse the html content using any parser 
soup = BeautifulSoup(html_content,"html.parser") 


In [ ]:
# The data we need are wrapped in <a> tags. Grab all the <a> tags here and we'll filter later.
datas = soup.find_all("a")

In [ ]:
k = []
v = []
for item in datas:
    if 'engine' in item['href']:
        k.append(item['href'])
        v.append(item.text)

In [ ]:
lookup_dict = {'URL': k, 'Engine': v}

In [ ]:
lookup_dict

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(lookup_dict)

In [ ]:
df

In [157]:
url_p1 = 'https://asn.flightsafety.org'
def lookup_ac(url_p2):
    return_list = []
    url = f'{url_p1}{url_p2}'
    sub_content = requests.get(url).text
    soup2 = BeautifulSoup(sub_content,"html.parser")
    u_list=soup2.find_all('ul')[1]
    i = 0 
    while i < len(u_list.select('li')):
        (u_list.select('li')[i].text)
        return_list.append(u_list.select('li')[i].text)
        i += 1
    return(return_list)

In [158]:
df['Models'] = df['URL'].map(lookup_ac)

In [155]:
#df = df.drop('Models', axis=1)

In [159]:
df

,URL,Engine,Models
0,/database/engine/AL-250,Allison 250,"[Britten-Norman BN-2 Islander, GAF Nomad]"
1,/database/engine/AL-501,Allison 501,"[Aero Spacelines Mini Guppy Turbine, Convair C..."
2,/database/engine/AL-2100,Allison AE2100,"[Lockheed C-130 Hercules, SAAB 2000]"
3,/database/engine/AL-3007,Allison AE3007,"[Cessna 750 Citation X, Embraer ERJ-135, Embra..."
4,/database/engine/AV-LEON,Alvis Leonides,[Scottish Aviation Twin Pioneer]
...,...,...,...
141,/database/engine/WR-R1820,Wright R-1820,"[Boeing B-17 Flying Fortress, Boeing S.307 Str..."
142,/database/engine/WR-R2600,Wright R-2600,"[Boeing 314, Chase YC-122 Avitruc, Consolidate..."
143,/database/engine/WR-R3350,Wright R-3350,"[Canadair CP-107 Argus, Douglas DC-7, Fairchil..."
144,/database/engine/WR-R975,Wright R-975,[Ford Tri-Motor]
